<a href="https://colab.research.google.com/github/vin-thomas/Enron_email_classification/blob/main/LSTM_Enron_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1.0 Import**

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import torch
import glob
from nltk.tokenize import word_tokenize
import torch.nn.functional as F
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AIML Labs/@Project/working files 2b/Pre_processed/enron_csv_1', index_col=[0])

In [ ]:
import re

add_fm = []
add_to = []
add_cc = []
add_bcc = []
for item in df.Raw_Text:
  fm_emails = re.findall(r"From: ([a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+)", str(item))
  to_emails = re.findall(r"To: ([a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+)", str(item))
  cc_emails = re.findall(r"Cc: ([a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+)", str(item))
  bcc_emails = re.findall(r"Bcc: ([a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+)", str(item))
  if fm_emails:
    add_fm.append(fm_emails[0])
  else:
    add_fm.append(1000)
  if to_emails:
    add_to.append(to_emails[0])
  else:
    add_to.append(1000)
  if cc_emails:
    add_cc.append(cc_emails[0])
  else:
    add_cc.append(1000)
  if bcc_emails:
    add_bcc.append(bcc_emails[0])
  else:
    add_bcc.append(1000)  

In [ ]:
df['From']= add_fm
df['To']= add_to
df['CC']= add_cc
df['BCC']= add_bcc

In [ ]:
from datetime import datetime

def preprocess_date(date):
  format_data = "%a, %d %b %Y"
  date = datetime.strptime(date, format_data)
  return date

In [ ]:
date_email =[]

for item in df.Raw_Text:
  date = re.findall(r"Date: ([a-zA-Z]{3}, \d+ [a-zA-Z]{3} \d{4})", str(item))
  date= preprocess_date(date[0])
  date_email.append(date)


In [ ]:
df['date']= date_email

##**Extracting Subject**

In [ ]:
item_list = []
for item in df.Raw_Text:
  item = re.sub(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+', '', item)
  item_list.append(item)

In [ ]:
len(item_list)

517401

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\\', '', text)
    text = re.sub(r'\\n', '', text)
    text = re.sub(r'-', '', text)
    text = re.sub(r'=', '', text)
    text = re.sub(r'/', '', text)
    return text

In [ ]:
subject_email = []

for item in item_list:
  subject = re.findall(r"Subject: ([^\\]*)", str(item))
  subject = preprocess_text(subject[0])
  word_list = subject.split(' ')
  if len(word_list)>10:
    word_list = word_list[:10]
  subject = " ".join(word_list)
  subject_email.append(subject)

In [ ]:
df['Subject']= subject_email

##**Extracting Body**

In [ ]:
body_email =[]
for item in item_list:
  body = re.findall (r"FileName:([\s\S]*)$", str(item))
  body = preprocess_text(body[0])
  word_list = body.split(' ')
  if len(word_list)>90:
    word_list = word_list[1:70]
  word_list = " ".join(word_list)
  body_email.append(word_list)

In [ ]:
df['Body']= body_email

In [ ]:
df_final = df.drop(['Raw_Text'], axis=1)
df_final= df_final.drop_duplicates(subset=['Body', 'From'])
df_final = df_final[df_final.Folder != 'all_documents']
df_final = df_final[df_final.Folder != 'deleted_items']
df_final = df_final[df_final.Body != 'na']
df_final['Text']= df_final['Body']+' '+ df_final['Subject']
df_final.drop(['Subject', 'Body'], axis=1, inplace= True)
df_final['from_id']= df_final['From'].factorize()[0]
df_final['to_id']= df_final['To'].factorize()[0]
df_final['cc_id']= df_final['CC'].factorize()[0]
df_final['bcc_id']= df_final['BCC'].factorize()[0]
df_final['folder']= df_final['Folder'].factorize()[0]
df_final['name']= df_final['Name'].factorize()[0]
df_final.drop(['Name', 'Folder', 'From', 'To', 'CC', 'BCC', 'date'], axis=1, inplace= True)
df_lstm= df_final
df_lstm= df_lstm.reset_index(drop=True)

df_lstm = df_lstm[["Text", "from_id", "to_id", "cc_id", "bcc_id", "name", "folder"]]
df_addl = df_lstm[['name', 'from_id', 'to_id', 'cc_id', 'bcc_id']]

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()  

df_addl = scaler.fit_transform(df_addl)

In [ ]:
df_addl.shape

(204850, 5)

In [ ]:
print(df_lstm.nunique())

Text       204678
from_id     15295
to_id       16054
cc_id        7095
bcc_id       6972
name          225
folder       1328
dtype: int64


In [ ]:
labels= df_lstm.folder
content = df_lstm.Text

In [ ]:
type(labels), len(labels), type(content), len(content)

(pandas.core.series.Series, 204850, pandas.core.series.Series, 204850)

In [ ]:
len(labels.unique())

1328

##**4.0 Prepare a word index, dictionary**

In [ ]:
word2idx= {}
idx = 0
n_l = []
news_len =0
i=0

for item in content:
  word_list = word_tokenize(str(item))
  n_l.append(len(word_list))
  for word in word_list:
    i+=1
    if word not in word2idx:
      word2idx[word]= idx
      idx += 1
      
print (i, idx)

15147377 465523


In [ ]:
df_addl.shape

(204850, 5)

In [ ]:
max_len_article = max(n_l)
no_of_articles= len(content)

In [ ]:
dataset = np.zeros((no_of_articles, 100), dtype= int)
dataset.shape

(204850, 100)

In [ ]:
i=0
idx_list=[]
for item in content:
  word_list= word_tokenize(str(item))
  for token in word_list:
    if word2idx.get(token) is not None:
      idx = word2idx.get(token)
    else:
      idx= 0
    idx_list.append(idx)
  pad_list = [0]*(max_len_article- len(idx_list))
  idx_list = idx_list + pad_list
  dataset[i]= idx_list[0:100]
  idx_list=[]
  i+=1
 


In [ ]:
dataset= torch.tensor(dataset)

In [ ]:
dataset.shape

torch.Size([204850, 100])

####**5.0 DownLoad the pretrained vectors**

In [ ]:
URL = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip"
FILE = "fastText"
!wget -P $FILE $URL
!unzip $FILE/crawl-300d-2M.vec.zip -d $FILE

--2022-02-08 16:09:15--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1.4G) [application/zip]
Saving to: ‘fastText/crawl-300d-2M.vec.zip’

crawl-300d-2M.vec.z 100%[===================>]   1.42G  20.1MB/s    in 74s     

2022-02-08 16:10:31 (19.5 MB/s) - ‘fastText/crawl-300d-2M.vec.zip’ saved [1523785255/1523785255]

Archive:  fastText/crawl-300d-2M.vec.zip
  inflating: fastText/crawl-300d-2M.vec  


###**6.0 Load the pre-trained vectors and create the embedding matrix**

In [ ]:
fin = open('/content/fastText/crawl-300d-2M.vec', 'r', encoding='utf-8', newline='\n', errors='ignore')
n, d = map(int, fin.readline().split())

In [ ]:
len(word2idx)

465523

In [ ]:
embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))

In [ ]:
count =0
for line in fin:
  tokens = line.rstrip().split(' ')
  word = tokens[0]
  if word in word2idx:
    embeddings[word2idx[word]] = np.array(tokens[1:], dtype=np.float32)

In [ ]:
embeddings= torch.tensor(embeddings)

Now we have an embedding array wich has the vector for each word in our dictionary. Further, we have 'dataset' which gives the word index for each article

###**7.0 Create Pytorch Dataloaders**

In [ ]:
labels, uniques = pd.factorize(labels)

In [ ]:
len(np.unique(labels))

1328

In [ ]:
labels = torch.from_numpy(labels)

In [ ]:
dataset.shape, labels.shape

(torch.Size([204850, 100]), torch.Size([204850]))

In [ ]:
dataset_= df_lstm.drop(['Text', 'folder'], axis=1)

In [ ]:
type(dataset), type(dataset_)

(torch.Tensor, pandas.core.frame.DataFrame)

In [ ]:
dataset= torch.Tensor(dataset_.values)

In [ ]:
df_addl= torch.Tensor(df_addl)

In [ ]:
type(df_addl)

torch.Tensor

In [ ]:
dataset.shape, labels.shape, df_addl.shape

(torch.Size([204850, 5]), torch.Size([204850]), torch.Size([204850, 5]))

In [ ]:
from sklearn.model_selection import train_test_split

# Train Test Split
train_inputs, val_inputs, train_labels, val_labels, train_addl, val_addl = train_test_split(dataset, labels, df_addl, test_size=0.1, random_state=42)

In [ ]:
len(train_inputs), len(val_inputs), type(train_inputs), type (val_inputs), type(dataset), type(labels)

(184365, 20485, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor)

In [ ]:
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,
                              SequentialSampler)


batch_size=100

# Create DataLoader for training data
train_data = TensorDataset(train_inputs, train_addl, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create DataLoader for validation data
val_data = TensorDataset(val_inputs, val_addl, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=300)


##**8.Device**

In [ ]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla K80


In [ ]:
len (labels.unique())

1328

##**8. LSTM Model Architecture**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class LSTM(nn.Module):
  def __init__(self,
               pretrained_embedding,
               ):

      super(LSTM, self).__init__()
      self.embedding = nn.Embedding.from_pretrained(pretrained_embedding, freeze=False)

      self.lstm= nn.LSTM(input_size= 300, hidden_size= 1600, num_layers=2, batch_first= True)

      self.fc1 = nn.Linear(1605, 1200)
      self.fc2 = nn.Linear(1200, len(labels.unique()))
              
    
  def forward(self, dataset, addl_features):

    x_embed = self.embedding(dataset).float()
    x, _ = self.lstm(x_embed)
    x = x[:, -1, :]
    x = torch.cat((x, addl_features), -1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    
    logits = x
    
    return logits



##**9. Instatiate the CNN Model**

In [ ]:
model = LSTM(embeddings)
model = model.to(device)
model

LSTM(
  (embedding): Embedding(465523, 300)
  (lstm): LSTM(300, 1600, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=1605, out_features=1200, bias=True)
  (fc2): Linear(in_features=1200, out_features=1328, bias=True)
)

##**9.  Optimizer and Loss Function**

In [ ]:
import torch.optim as optim

# Instantiate Adadelta optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=.01)
#optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
criterion = torch.nn.CrossEntropyLoss()

##**10. Train**

In [ ]:
# No of Epochs
epoch = 10

# keeping the network in train mode
model.train()
train_losses,  train_accuracy = [], []

# Loop for no of epochs
for e in range(epoch):
    train_loss = 0
    correct = 0
    # Iterate through all the batches in each epoch
    i=0
    for inputs, addl_inputs, labels in train_dataloader:
      i+=1
      # Convert the image and label to gpu for faster execution
      inputs = inputs.to(device).to(torch.int64)
      addl_inputs= addl_inputs.to(device).to(torch.int64)
       
      labels = labels.to(device)
          
      # Zero the parameter gradients
      optimizer.zero_grad()
      
      # Passing the data to the model (Forward Pass)
      outputs = model(inputs, addl_inputs)

                
      # Calculating the loss
      loss = criterion(outputs, labels)
      train_loss += loss.item()

      # Performing backward pass (Backpropagation)
      loss.backward()

      # optimizer.step() updates the weights accordingly
      optimizer.step()

      _, predicted = torch.max(outputs, 1)
      correct += (predicted == labels).sum().item()
      #print(i, ')', loss.item(), (predicted == labels).sum().item())
      
     
    # Accuracy calculation
    
    train_losses.append(train_loss/len (train_data))
    train_accuracy.append(100 * correct/len(train_data))
    print('epoch: {}, Train Loss:{:.6f} Train Accuracy: {:.2f}'.format(e+1,train_losses[-1], train_accuracy[-1]))

epoch: 1, Train Loss:0.022656 Train Accuracy: 49.86
epoch: 2, Train Loss:0.017321 Train Accuracy: 61.28
epoch: 3, Train Loss:0.016665 Train Accuracy: 61.81
epoch: 4, Train Loss:0.015844 Train Accuracy: 63.19
epoch: 5, Train Loss:0.015423 Train Accuracy: 63.89
epoch: 6, Train Loss:0.015252 Train Accuracy: 64.04
epoch: 7, Train Loss:0.015142 Train Accuracy: 64.12
epoch: 8, Train Loss:0.015042 Train Accuracy: 64.24
epoch: 9, Train Loss:0.014849 Train Accuracy: 64.68
epoch: 10, Train Loss:0.014896 Train Accuracy: 64.40


In [ ]:

eval_model = model
eval_model.eval
val_accuracy = []
val_loss = []

for inputs, addl_inputs, labels in val_dataloader:
  inputs = inputs.to(device).to(torch.int64)
  addl_inputs= addl_inputs.to(device).to(torch.int64)
  labels = labels.to(device)
  logits = eval_model(inputs, addl_inputs)
  print(type(logits), logits.shape)

  loss = criterion (logits, labels)
  val_loss.append(loss.item())
  _, predicted = torch.max(logits, 1)

    
  accuracy = (predicted == labels).cpu().numpy().mean() * 100

  val_accuracy.append(accuracy)

val_loss = np.mean(val_loss)
val_accuracy = np.mean(val_accuracy)

print (val_loss, val_accuracy)


<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch.Tensor'> torch.Size([300, 1328])
<class 'torch